In [1]:
# IMPORT NECESSARY LIBRARIES
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from imutils.video import VideoStream
import numpy as np
import imutils
import time
import cv2
import os
import face_recognition
from datetime import datetime, date
import win32com.client

# SYSTEM VOICE
system = win32com.client.Dispatch("SAPI.SpVoice")

flag = 0
flag1 = 0

# GET LIST OF IMAGES WITH THEIR NAMES FROM LOCATION
databasePath = 'attendanceDatabase'

# ATTENDANCE SHEET PATH
attSheetPath = 'attendanceSheets'

# IMAGES WILL BE LOADED IN THIS LIST
images = []

# NAMES OF STUDENTS/STAFF
classNames = []

peepList = os.listdir(databasePath)
print(f'DATABASE PEOPLE ENCODINGS: \n{peepList}')



for peeps in peepList:  # IMPORT IMAGES ONE BY ONE
    curImg = cv2.imread(f'{databasePath}/{peeps}')
    images.append(curImg)  # append the current image
    classNames.append(os.path.splitext(peeps)[0])  # names of images


# GET ENCODINGS OF SAVED IMAGES
def find_encodings(images):
    encode_lsit = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encode_lsit.append(encode)
    return encode_lsit


encods = find_encodings(images)

print('\nENCODING COMPLETE')


def attendance(img, label):
    # MARK ATTENDANCE INTO FILE
    dateToday = date.today()

    def frst(name, now):
        
        global flag1

        # READS N WRITES ON ATTENDANCE FILE
        with open(f'{attSheetPath}/attendance_{dateToday}.csv', 'r+') as f:
            # READS FOR EXISTING FILE
            myDatalist = f.readlines()
            # EXISTING NAMES ARE APPENDED
            nameList = []
            for line in myDatalist:
                entry = line.split(',')
                nameList.append(entry[0])
            
            # IF PRESENT PERSON IS NOT IN ATTENDANCE SHEET, THEN IT UPDATES
            if name not in nameList:
                timeNow = now.strftime('%H:%M:%S')
                f.writelines(f'\n{name},{dateToday},{timeNow}')
                system.Speak(f"{name}'s ATTENDANCE MARKED")
            else:
                flag1+=1
                if flag1%3 == 0:
                    system.Speak(f"{name} YOUR ATTENDANCE ALREADY MARKED")
                    print(f"{name} YOUR ATTENDANCE ALREADY MARKED!!!\n\n")

    def scnd(name, now):
        with open(f'{attSheetPath}/attendance_{dateToday}.csv', 'w') as f:  # r+ is for reading and writing
            f.writelines(f'Name,Date,Time')
        frst(name, now)

    def markSheetAttendance(name, now):

        # IF THE FILE ALREADY EXISTS
        if os.path.isfile(f'{attSheetPath}/attendance_{dateToday}.csv'):
            frst(name, now)
        else:
            # IF NOT THEN CREATE IT
            scnd(name, now)

    encodelistforknownfaces = encods


    # READ INPUT IMAGE
    imgS = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # FINDS FACES IN INPUT IMAGE
    curFace = face_recognition.face_locations(imgS, model='hog')

    # ENCODINGS OF CURRENT FACE
    encodeCurFace = face_recognition.face_encodings(img, curFace)

    # zip IS USED TO GET ENCODINGS AND FACE IN SAME LOOP
    for encodeFace, faceLoc in zip(encodeCurFace, curFace):

        # COMPARE INPUT FACE WITH DATA
        matches = face_recognition.compare_faces(encodelistforknownfaces, encodeFace)
        # CHECKS FOR DISTACE AMONG THEM
        faceDis = face_recognition.face_distance(encodelistforknownfaces, encodeFace)
        #print(faceDis)
        # TAKES LOWEST INDEX DISTANCE
        matchIndex = np.argmin(faceDis)

        # IF THE PERSON ALREADY IN DATA THEN MARK ATTENDANCE
        if matches[matchIndex]:
            now = datetime.now()
            name = classNames[matchIndex].upper()
            name = name.split('_')[0]
            print(f'Recognised Person --> {name}\n')
            markSheetAttendance(name, now)


# DETECT FOR MASK OR NO MASK
def detect_and_predict_mask(frame, faceNet, maskNet):
    # GRAB DIMENSIONS OF INPUT IMAGE
    (h, w) = frame.shape[:2]

    # CONSTRUCT A BLOB
    blob = cv2.dnn.blobFromImage(image=frame, scalefactor=1.0, size=(224, 224), mean=(104.0, 177.0, 123.0))

    # PASS THE BLOB THROUGH THE NETWORK TO DETECT FACE
    faceNet.setInput(blob)
    detections = faceNet.forward()

    # INITIALIZE LIST OF FACES AND LOCATIONS
    faces = []
    locs = []
    # MASK PREDICTIONS
    preds = []

    # LOOP OVER FOR DETECTIONS
    for i in range(0, detections.shape[2]):

        # CONFIDENCE ASSOCIATED WITH DETECTIONS
        confidence = detections[0, 0, i, 2]

        # FILTERING MASK DETECTIONS WITH MIN THRESHOLD CONFIDENCE
        if confidence > 0.5:
            # BOUNDING BOX FOR FACE
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # BOUNDING BOX WITHIN THE FRAME
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))

            # PREPROCESSING THE INPUT
            face = frame[startY:endY, startX:endX]
            face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
            face = cv2.resize(face, (224, 224))
            face = img_to_array(face)
            face = preprocess_input(face)

            faces.append(face)
            locs.append((startX, startY, endX, endY))

    # PREDICTION ONLY MIN ONE FACE DETECTED
    if len(faces) > 0:
        faces = np.array(faces, dtype="float32")
        #print(faces.shape)
        preds = maskNet.predict(faces, batch_size=32)
        #print(preds)
    return (locs, preds)


# LOADING PRETRAINED FACE DETECTING MODELS
prototxtPath = r"face_detector\deploy.prototxt"
weightsPath = r"face_detector\res10_300x300_ssd_iter_140000.caffemodel"
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

# LOADING THE MASK DETECTING MODEL
maskNet = load_model("mask_model.h5")

#STARTING LIVE VIDEO STREAM
print("\n\n------------------STARTING LIVE VIDEO STREAM------------------\n\n")
vs = VideoStream(src=0).start()

while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=800)

    # CHECKING FOR MASK ONN OR OFF
    (locs, preds) = detect_and_predict_mask(frame, faceNet, maskNet)

    for (box, pred) in zip(locs, preds):
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred
        print(f'Mask: {mask}\nWithout Mask: {withoutMask}\n')

        # LABELING THE VIDEO STREAM
        label = "Mask" if mask > withoutMask else "No Mask"
        
        if label == 'Mask':
            flag+=1
            cv2.putText(frame, 'PLEASE REMOVE MASK ', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_4)
            cv2.putText(frame, 'FOR ATTENDANCE!!!', (50, 80), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_4)
            if flag%25 == 0:
                system.Speak("PLEASE REMOVE MASK FOR ATTENDANCE")
                
        else:
            attendance(frame, label)
            

        color = (0, 0, 255) if label == "Mask" else (0, 255, 0)

        # MASK PROBABILITY
        label = f"{label} {max(mask, withoutMask) * 100}"

        cv2.putText(frame, label, (startX, startY - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)

    # OUTPUT WINDOW
    cv2.imshow("LIVE CAPTURE", frame)
    key = cv2.waitKey(1) & 0xFF

    if key == ord("q"):
        break

cv2.destroyAllWindows()
vs.stop()

DATABASE PEOPLE ENCODINGS: 
['billgates_1.jpg', 'billgates_2.jpg', 'billgates_3.jpg', 'chavi.jpeg', 'elon_1.jfif', 'elon_2.jfif', 'elon_3.jpg', 'elon_4.jpg', 'harsh.jpeg', 'parakh_1.jpeg', 'parakh_2.jpeg', 'parakh_3.jpeg']

ENCODING COMPLETE


------------------STARTING LIVE VIDEO STREAM------------------


Mask: 1.0
Without Mask: 5.703067351703339e-09

Mask: 1.0
Without Mask: 2.3401145465840045e-09

Mask: 1.0
Without Mask: 1.682477268616367e-08

Mask: 1.0
Without Mask: 2.292237866186042e-08

Mask: 1.0
Without Mask: 7.4393513571635594e-09

Mask: 1.0
Without Mask: 8.099418913332102e-09

Mask: 1.0
Without Mask: 2.8285644937398047e-09

Mask: 1.0
Without Mask: 7.51357553951948e-09

Mask: 1.0
Without Mask: 4.615059445711722e-09

Mask: 1.0
Without Mask: 2.7186493056774452e-08

Mask: 1.0
Without Mask: 4.357490812623155e-09

Mask: 1.0
Without Mask: 1.5936112429670857e-09

Mask: 1.0
Without Mask: 8.001481699437818e-10

Mask: 1.0
Without Mask: 3.2956244400139667e-09

Mask: 1.0
Without Mask: 4.03

PermissionError: [Errno 13] Permission denied: 'attendanceSheets/attendance_2021-05-21.csv'

In [1]:
# ATTENDANCE SHEET PATH
attSheetPath = 'attendanceSheets'

lst1 = ['chavi', 'parakh', 'harsh','ram', 'shyam', 'billgates']
lst2 = [0,0,0,0,0,0]
import glob
import pandas as pd
flag=0

for file_name in glob.glob(f'{attSheetPath}/*.csv'):
    flag +=1
    print(file_name)
    x = pd.read_csv(file_name, low_memory=False)

    for i in x['Name']:
        for j in range(len(lst1)):
            if i == lst1[j].upper():
                lst2[j]+=1
print(f'\nTotal Files Count: {flag}\n')
print(lst2)
    
                

attendanceSheets\attendance_2021-04-27.csv
attendanceSheets\attendance_2021-04-28.csv
attendanceSheets\attendance_2021-04-29.csv
attendanceSheets\attendance_2021-05-05.csv
attendanceSheets\attendance_2021-05-11.csv
attendanceSheets\attendance_2021-05-13.csv
attendanceSheets\attendance_2021-05-14.csv
attendanceSheets\attendance_2021-05-15.csv
attendanceSheets\attendance_2021-05-16.csv
attendanceSheets\attendance_2021-05-17.csv
attendanceSheets\attendance_2021-05-18.csv
attendanceSheets\attendance_2021-05-20.csv
attendanceSheets\x.csv
attendanceSheets\x2.csv
attendanceSheets\x3.csv
attendanceSheets\x4.csv
attendanceSheets\x5.csv
attendanceSheets\x6.csv

Total Files Count: 18

[1, 17, 0, 0, 0, 2]


In [3]:
for a,b in zip(lst1,lst2):
    print(f"{a.upper()}     \t{b} / {flag}  \t{b/flag*100}%")

CHAVI     	1 / 18  	5.555555555555555%
PARAKH     	17 / 18  	94.44444444444444%
HARSH     	0 / 18  	0.0%
RAM     	0 / 18  	0.0%
SHYAM     	0 / 18  	0.0%
BILLGATES     	2 / 18  	11.11111111111111%
